
TIME SERIES NORMALIZATION

Input:

1. 'a': numpy array contenente le time series
2. asse su sui calcolare media e deviazione standard
3. ddf: delta degree of freedom. =0 deviazione standard della popolazione


Output:

1. normalized array of time series

In [ ]:
     
def zscore(a, axis=0, ddof=0):
    a = np.asanyarray(a) #convert the input a in an array
    mns = a.mean(axis=axis) #compute the mean of the array
    sstd = a.std(axis=axis, ddof=ddof) #compute the standard deviation of the array

    if axis and mns.ndim < a.ndim: #
        res = ((a - np.expand_dims(mns, axis=axis)) /
               np.expand_dims(sstd, axis=axis))
    else:
        res = (a - mns) / sstd

    return np.nan_to_num(res)

a=no
zscore()    
